In [2]:
import pandas as pd
import pickle

In [3]:
def save_to_pickle(pkl_path,dict_obj):
    file = open(pkl_path, "wb")
    pickle.dump(dict_obj, file)
    file.close()

In [4]:
def map_feature_label(target_path,dict_csv_path):
    all_feature=pd.read_csv(dict_csv_path)
    feature_label_divtionary={k:v for k,v in zip(all_feature['feature_name'],all_feature['label'])}
    item_feature=pd.read_csv(target_path)
    item_feature['feature_category_id'] = item_feature['feature_category_id'].apply(str)
    item_feature['feature_value_id'] = item_feature['feature_value_id'].apply(str)
    item_feature['feature_name']=item_feature["feature_category_id"] + "_" + item_feature["feature_value_id"]
    item_feature['lebal']=[feature_label_divtionary[i] for i in item_feature['feature_name']]
    item_feature['lebal'].apply(int)
    return item_feature

# 計算item_category+item_value之出現次數

In [9]:
item_feature=pd.read_csv("item_features.csv")
# item_feature.columns
all_feature=item_feature.groupby(['feature_category_id','feature_value_id']).count()
# all_feature
all_feature.to_csv('__feature.csv')
all_feature=pd.read_csv('__feature.csv')
all_feature

,feature_category_id,feature_value_id,item_id
0,1,60,1
1,1,143,10
2,1,358,2
3,1,461,924
4,1,517,2
...,...,...,...
899,72,751,55
900,72,829,405
901,72,883,65
902,73,91,2381


# Data preprocessing
- item=>vector
- session=>vector
## feature encoding

In [6]:
all_feature=all_feature.drop(columns=['item_id'])

all_feature['feature_category_id'] = all_feature['feature_category_id'].apply(str)
all_feature['feature_value_id'] = all_feature['feature_value_id'].apply(str)
all_feature['feature_name']=all_feature["feature_category_id"] + "_" + all_feature["feature_value_id"]
all_feature['label']=[i for i in range(all_feature.shape[0])]
# all_feature=all_feature.drop(columns=['feature_category_id','feature_value_id'])
all_feature

,feature_category_id,feature_value_id,feature_name,label
0,1,60,1_60,0
1,1,143,1_143,1
2,1,358,1_358,2
3,1,461,1_461,3
4,1,517,1_517,4
...,...,...,...,...
899,72,751,72_751,899
900,72,829,72_829,900
901,72,883,72_883,901
902,73,91,73_91,902


In [7]:
all_feature.to_csv('all_feature_encoding.csv')

## item==>vetor
### 1. mapping

In [40]:
item_feature=map_feature_label('item_features.csv','all_feature_encoding.csv')
item_feature

,item_id,feature_category_id,feature_value_id,feature_name,lebal
0,2,56,365,56_365,637
1,2,62,801,62_801,720
2,2,68,351,68_351,796
3,2,33,802,33_802,378
4,2,72,75,72_75,878
...,...,...,...,...,...
471746,28143,68,351,68_351,796
471747,28143,55,390,55_390,578
471748,28143,11,109,11_109,131
471749,28143,73,91,73_91,902


### 2.item==>vetor

In [25]:
res_item_vector={}
temp=list(set(item_feature['item_id']))
for i in range(len(temp)):
    now_item=temp[i]
    now_vector=[0 for i in range(904)]
    for index, row in item_feature[item_feature['item_id']==a[i]].iterrows():
        now_vector[row['lebal']]=1
        res_item_vector[now_item]=now_vector
    res_item_vector[row['item_id']]=now_vector
save_to_pickle("item_vector.pkl",res_item_vector)

In [30]:
# res_item_vector.keys()
sum(res_item_vector[2])

13

## train_sessions => vector

In [8]:
import os
def session2vec(csv_path):
    data=pd.read_csv(csv_path)
    res_session_vector={}
    temp=sorted(list(set(data['session_id'])))
    for i in range(len(temp)):
        now_sess=temp[i]
        now_vector=[]
        for index, row in data[data['session_id']==temp[i]].iterrows():
            now_vector=now_vector+res_item_vector[row['item_id']]
        res_session_vector[row['session_id']]=now_vector
#     print(res_session_vector.keys())
    save_to_pickle(csv_path.split('.')[0]+'.pkl',res_session_vector)
session2vec('test_final_sessions.csv')
session2vec('test_leaderboard_sessions.csv')
session2vec('train_purchases.csv')
session2vec('train_sessions.csv')

# reference
- [How to iterate over rows in a DataFrame in Pandas](https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas)
-[How to save a dictionary to a file in Python](https://www.adamsmith.haus/python/answers/how-to-save-a-dictionary-to-a-file-in-python)